# 处理转录拼写错误：提示 vs 后处理我们正在解决提高转录准确性的问题，特别是在涉及公司名称和产品引用时。我们的解决方案涉及使用Whisper提示参数和GPT-4的后处理能力的双重策略。纠正不准确性的两种方法是：- 我们直接将正确拼写的列表输入Whisper的提示参数，以指导初始转录。- 我们利用GPT-4在转录后修复拼写错误，再次使用相同的正确拼写列表在提示中。这些策略旨在确保对不熟悉的专有名词进行精确转录。## 设置要开始，请：- 导入OpenAI Python库（如果没有，请使用```pip install openai```进行安装）- 下载音频文件示例

In [1]:
# 导入from openai import OpenAI  # 用于进行OpenAI API调用import urllib  # 用于下载示例音频文件import os  # 用于访问环境变量client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "<your OpenAI API key if not set as env var>"))

In [3]:
# 设置下载路径ZyntriQix_remote_filepath = "https://cdn.openai.com/API/examples/data/ZyntriQix.wav"# 设置本地保存位置ZyntriQix_filepath = "data/ZyntriQix.wav"# 下载示例音频文件并保存到本地urllib.request.urlretrieve(ZyntriQix_remote_filepath, ZyntriQix_filepath)

('data/ZyntriQix.wav', <http.client.HTTPMessage at 0x10559a910>)

## 通过虚构的音频录制来设定我们的基准线我们的参考点是一个独白，这个独白是由ChatGPT根据作者给出的提示生成的。然后作者将这段内容朗读出来。因此，作者既通过提示引导了ChatGPT的输出，又通过朗读将其变得生动起来。

我们的虚构公司ZyntriQix提供各种科技产品。这些产品包括Digique Plus，CynapseFive，VortiQore V8，EchoNix Array，OrbitalLink Seven和DigiFractal Matrix。我们还发起了几项倡议，如PULSE，RAPT，B.R.I.C.K.，Q.U.A.R.T.Z.和F.L.I.N.T.。

In [10]:
# 定义一个包装函数，用于观察提示如何影响转录结果def transcribe(prompt: str, audio_filepath) -> str:    """根据提示，将音频文件转录出来。"""    transcript = client.audio.transcriptions.create(        file=open(audio_filepath, "rb"),        model="whisper-1",        prompt=prompt,    )    return transcript.text

In [11]:
# 无提示的基础转录transcribe(prompt="", audio_filepath=ZyntriQix_filepath)

"Have you heard of ZentricX? This tech giant boasts products like Digi-Q+, Synapse 5, VortiCore V8, Echo Nix Array, and not to forget the latest Orbital Link 7 and Digifractal Matrix. Their innovation arsenal also includes the Pulse framework, Wrapped system, they've developed a brick infrastructure court system, and launched the Flint initiative, all highlighting their commitment to relentless innovation. ZentricX, in just 30 years, has soared from a startup to a tech titan, serving us tech marvels alongside a stimulating linguistic challenge. Quite an adventure, wouldn't you agree?"

Whisper错误地转录了我们公司的名称、产品名称，并且错误地将首字母大写的缩写。让我们在提示中传递正确的名称列表。

In [12]:
# 将正确的拼写名称添加到提示中transcribe(    prompt="ZyntriQix, Digique Plus, CynapseFive, VortiQore V8, EchoNix Array, OrbitalLink Seven, DigiFractal Matrix, PULSE, RAPT, B.R.I.C.K., Q.U.A.R.T.Z., F.L.I.N.T.",    audio_filepath=ZyntriQix_filepath,)

"Have you heard of ZyntriQix? This tech giant boasts products like Digique Plus, CynapseFive, VortiQore V8, EchoNix Array, and not to forget the latest OrbitalLink Seven and DigiFractal Matrix. Their innovation arsenal also includes the PULSE framework, RAPT system. They've developed a B.R.I.C.K. infrastructure, Q.U.A.R.T. system, and launched the F.L.I.N.T. initiative, all highlighting their commitment to relentless innovation. ZyntriQix in just 30 years has soared from a startup to a tech titan, serving us tech marvels alongside a stimulating linguistic challenge. Quite an adventure, wouldn't you agree?"

当传递产品名称列表时，一些产品名称被正确转录，而另一些仍然拼写错误。

In [13]:
# 请在提示中添加完整的产品清单。transcribe(    prompt="ZyntriQix, Digique Plus, CynapseFive, VortiQore V8, EchoNix Array, OrbitalLink Seven, DigiFractal Matrix, PULSE, RAPT, AstroPixel Array, QuantumFlare Five, CyberPulse Six, VortexDrive Matrix, PhotonLink Ten, TriCircuit Array, PentaSync Seven, UltraWave Eight, QuantumVertex Nine, HyperHelix X, DigiSpiral Z, PentaQuark Eleven, TetraCube Twelve, GigaPhase Thirteen, EchoNeuron Fourteen, FusionPulse V15, MetaQuark Sixteen, InfiniCircuit Seventeen, TeraPulse Eighteen, ExoMatrix Nineteen, OrbiSync Twenty, QuantumHelix TwentyOne, NanoPhase TwentyTwo, TeraFractal TwentyThree, PentaHelix TwentyFour, ExoCircuit TwentyFive, HyperQuark TwentySix, B.R.I.C.K., Q.U.A.R.T.Z., F.L.I.N.T.",    audio_filepath=ZyntriQix_filepath,)

"Have you heard of ZentricX? This tech giant boasts products like DigiCube Plus, Synapse 5, VortiCore V8, EchoNix Array, and not to forget the latest Orbital Link 7 and Digifractal Matrix. Their innovation arsenal also includes the PULSE framework, RAPT system. They've developed a brick infrastructure court system and launched the F.L.I.N.T. initiative, all highlighting their commitment to relentless innovation. ZentricX in just 30 years has soared from a startup to a tech titan, serving us tech marvels alongside a stimulating linguistic challenge. Quite an adventure, wouldn't you agree?"

## 您可以使用GPT-4来纠正拼写错误利用GPT-4在语音内容事先未知且我们有产品名称列表可用时尤为有用。使用GPT-4的后处理技术比仅依赖Whisper的提示参数更具可扩展性，因为Whisper的提示参数有一个令牌限制为244。GPT-4允许我们处理更大的正确拼写列表，使其成为处理大量产品列表的更强大方法。然而，这种后处理技术并非没有局限性。它受所选模型的上下文窗口限制，当处理大量独特术语时可能会面临挑战。例如，拥有数千种SKU的公司可能会发现GPT-4的上下文窗口不足以满足其需求，他们可能需要探索替代解决方案。有趣的是，GPT-4的后处理技术似乎比仅使用Whisper更可靠。这种利用产品列表的方法增强了我们结果的可靠性。然而，这种增加的可靠性是有代价的，因为使用这种方法可能会增加成本并导致更高的延迟。

In [19]:
# 定义一个包装函数，用于观察提示如何影响转录结果def transcribe_with_spellcheck(system_message, audio_filepath):    completion = client.chat.completions.create(        model="gpt-4",        temperature=0,        messages=[            {"role": "system", "content": system_message},            {                "role": "user",                "content": transcribe(prompt="", audio_filepath=audio_filepath),            },        ],    )    return completion.choices[0].message.content

现在，让我们将原始产品列表输入到GPT-4中，并评估其性能。通过这样做，我们的目标是评估AI模型在没有关于要出现在转录中的确切术语的先前知识的情况下，正确拼写专有产品名称的能力。在我们的实验中，GPT-4成功地正确拼写了我们的产品名称，证实了它作为确保转录准确性的可靠工具的潜力。

In [20]:
system_prompt = "You are a helpful assistant for the company ZyntriQix. Your task is to correct any spelling discrepancies in the transcribed text. Make sure that the names of the following products are spelled correctly: ZyntriQix, Digique Plus, CynapseFive, VortiQore V8, EchoNix Array, OrbitalLink Seven, DigiFractal Matrix, PULSE, RAPT, B.R.I.C.K., Q.U.A.R.T.Z., F.L.I.N.T."new_text = transcribe_with_spellcheck(system_prompt, audio_filepath=ZyntriQix_filepath)print(new_text)

Have you heard of ZyntriQix? This tech giant boasts products like Digique Plus, CynapseFive, VortiQore V8, EchoNix Array, and not to forget the latest OrbitalLink Seven and DigiFractal Matrix. Their innovation arsenal also includes the PULSE framework, RAPT system, they've developed a B.R.I.C.K. infrastructure court system, and launched the F.L.I.N.T. initiative, all highlighting their commitment to relentless innovation. ZyntriQix, in just 30 years, has soared from a startup to a tech titan, serving us tech marvels alongside a stimulating linguistic challenge. Quite an adventure, wouldn't you agree?


在这种情况下，我们提供了一个包含所有先前使用的拼写以及额外新名称的全面产品列表。这种情况模拟了现实生活中的情况，我们有一个庞大的SKU列表，并且对出现在转录中的确切术语不确定。将这个庞大的产品名称列表输入系统，最终得到了正确的转录输出。

In [21]:
system_prompt = "You are a helpful assistant for the company ZyntriQix. Your task is to correct any spelling discrepancies in the transcribed text. Make sure that the names of the following products are spelled correctly: ZyntriQix, Digique Plus, CynapseFive, VortiQore V8, EchoNix Array,  OrbitalLink Seven, DigiFractal Matrix, PULSE, RAPT, AstroPixel Array, QuantumFlare Five, CyberPulse Six, VortexDrive Matrix, PhotonLink Ten, TriCircuit Array, PentaSync Seven, UltraWave Eight, QuantumVertex Nine, HyperHelix X, DigiSpiral Z, PentaQuark Eleven, TetraCube Twelve, GigaPhase Thirteen, EchoNeuron Fourteen, FusionPulse V15, MetaQuark Sixteen, InfiniCircuit Seventeen, TeraPulse Eighteen, ExoMatrix Nineteen, OrbiSync Twenty, QuantumHelix TwentyOne, NanoPhase TwentyTwo, TeraFractal TwentyThree, PentaHelix TwentyFour, ExoCircuit TwentyFive, HyperQuark TwentySix, GigaLink TwentySeven, FusionMatrix TwentyEight, InfiniFractal TwentyNine, MetaSync Thirty, B.R.I.C.K., Q.U.A.R.T.Z., F.L.I.N.T. Only add necessary punctuation such as periods, commas, and capitalization, and use only the context provided."new_text = transcribe_with_spellcheck(system_prompt, audio_filepath=ZyntriQix_filepath)print(new_text)

Have you heard of ZyntriQix? This tech giant boasts products like Digique Plus, CynapseFive, VortiQore V8, EchoNix Array, and not to forget the latest OrbitalLink Seven and DigiFractal Matrix. Their innovation arsenal also includes the PULSE framework, RAPT system, they've developed a B.R.I.C.K. infrastructure court system, and launched the F.L.I.N.T. initiative, all highlighting their commitment to relentless innovation. ZyntriQix, in just 30 years, has soared from a startup to a tech titan, serving us tech marvels alongside a stimulating linguistic challenge. Quite an adventure, wouldn't you agree?


我们正在使用GPT-4作为拼写检查器，使用与之前在提示中使用的相同的正确拼写列表。

In [22]:
system_prompt = "You are a helpful assistant for the company ZyntriQix. Your first task is to list the words that are not spelled correctly according to the list provided to you and to tell me the number of misspelled words. Your next task is to insert those correct words in place of the misspelled ones. List: ZyntriQix, Digique Plus, CynapseFive, VortiQore V8, EchoNix Array,  OrbitalLink Seven, DigiFractal Matrix, PULSE, RAPT, AstroPixel Array, QuantumFlare Five, CyberPulse Six, VortexDrive Matrix, PhotonLink Ten, TriCircuit Array, PentaSync Seven, UltraWave Eight, QuantumVertex Nine, HyperHelix X, DigiSpiral Z, PentaQuark Eleven, TetraCube Twelve, GigaPhase Thirteen, EchoNeuron Fourteen, FusionPulse V15, MetaQuark Sixteen, InfiniCircuit Seventeen, TeraPulse Eighteen, ExoMatrix Nineteen, OrbiSync Twenty, QuantumHelix TwentyOne, NanoPhase TwentyTwo, TeraFractal TwentyThree, PentaHelix TwentyFour, ExoCircuit TwentyFive, HyperQuark TwentySix, GigaLink TwentySeven, FusionMatrix TwentyEight, InfiniFractal TwentyNine, MetaSync Thirty, B.R.I.C.K., Q.U.A.R.T.Z., F.L.I.N.T."new_text = transcribe_with_spellcheck(system_prompt, audio_filepath=ZyntriQix_filepath)print(new_text)

The misspelled words are: ZentricX, Digi-Q+, Synapse 5, VortiCore V8, Echo Nix Array, Orbital Link 7, Digifractal Matrix, Pulse, Wrapped, brick, Flint, and 30. The total number of misspelled words is 12.

The corrected paragraph is:

Have you heard of ZyntriQix? This tech giant boasts products like Digique Plus, CynapseFive, VortiQore V8, EchoNix Array, and not to forget the latest OrbitalLink Seven and DigiFractal Matrix. Their innovation arsenal also includes the PULSE framework, RAPT system, they've developed a B.R.I.C.K. infrastructure court system, and launched the F.L.I.N.T. initiative, all highlighting their commitment to relentless innovation. ZyntriQix, in just MetaSync Thirty years, has soared from a startup to a tech titan, serving us tech marvels alongside a stimulating linguistic challenge. Quite an adventure, wouldn't you agree?
